In [2]:
import pandas as pd
import numpy as np
import hashlib
import re
import random as rd
import datetime
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
from similarity.levenshtein import Levenshtein
from similarity.normalized_levenshtein import NormalizedLevenshtein

import sys
sys.path.append('../')

from collections import Counter
import matplotlib.pyplot as plt

In [3]:
from importlib import reload
import EmbDI.data_preprocessing as dp

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/spoutnik23/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
reload(dp)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/spoutnik23/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'EmbDI.data_preprocessing' from '../EmbDI/data_preprocessing.py'>

#### Loading the first dataset

In [6]:
df1

,id,Beer_Name,Brew_Factory_Name,Style,ABV
0,0,TrÃ ¶ egs Nugget Nectar,TrÃ ¶ egs Brewing Company,American Amber / Red Ale,7.50 %
1,1,Fat Tire Amber Ale,New Belgium Brewing,American Amber / Red Ale,5.20 %
2,2,Levitation Ale,Stone Brewing Co. .,American Amber / Red Ale,4.40 %
3,3,Green Flash Hop Head Red Ale,Green Flash Brewing Co. .,American Amber / Red Ale,8.10 %
4,4,TrÃ ¶ egs Hopback Amber Ale,TrÃ ¶ egs Brewing Company,American Amber / Red Ale,6.00 %
...,...,...,...,...,...
4340,4340,Nonstop Forever,The Brew Gentlemen Beer Co. .,American Barleywine,9.00 %
4341,4341,Old Familiar Barleywine,Carolina Brewery,American Barleywine,8.30 %
4342,4342,Little Willie Barleywine,The Blue Ridge Brewing Co. .,American Barleywine,11.50 %
4343,4343,Olde Reliable,Laurelwood Public House & Brewery,American Barleywine,10.00 %


In [5]:
f1 = 'pipeline/experiments/beer/exp_data/tableA.csv'
df1 = pd.read_csv('../' + f1, encoding='utf-8')
print(df1.shape)
df1.head(5)

(4345, 5)


,id,Beer_Name,Brew_Factory_Name,Style,ABV
0,0,TrÃ ¶ egs Nugget Nectar,TrÃ ¶ egs Brewing Company,American Amber / Red Ale,7.50 %
1,1,Fat Tire Amber Ale,New Belgium Brewing,American Amber / Red Ale,5.20 %
2,2,Levitation Ale,Stone Brewing Co. .,American Amber / Red Ale,4.40 %
3,3,Green Flash Hop Head Red Ale,Green Flash Brewing Co. .,American Amber / Red Ale,8.10 %
4,4,TrÃ ¶ egs Hopback Amber Ale,TrÃ ¶ egs Brewing Company,American Amber / Red Ale,6.00 %


#### Loading the second dataset

In [5]:
f2 = 'pipeline/experiments/beer/exp_data/tableB.csv'
df2 = pd.read_csv('../' + f2, encoding='utf-8')
print(df2.shape)
df2.head(5)

(3000, 5)


,id,Beer_Name,Brew_Factory_Name,Style,ABV
0,0,Great Lakes Nosferatu,Great Lakes Brewing &#40; Ohio &#41;,American Strong Ale,8 %
1,1,4 Hands Reprise Centennial Red Ale,4 Hands Brewing Company,Amber Ale,6 %
2,2,10 Barrel Backdraft IPA,10 Barrel Brewing &#40; AB InBev &#41;,India Pale Ale &#40; IPA &#41;,6.50 %
3,3,Gore Range Fly Fisher Red Ale,Gore Range Brewery,Amber Ale,-
4,4,Naked Dove Nice & Naughty Christmas Ale,Naked Dove Brewing,Stout,5.50 %


#### Setting the Data Preprocessing parameters

In [66]:
parameters = {
    'output_file': 'beer-master',
    'concatenate': 'outer',
    'missing_value': 'nan,ukn,none,unknown,-,',
    'missing_value_strategy': '',
    'round_number': 1,
    'round_columns': 'abv',
    'auto_merge': False,
    'split_columns': '',
    'expand_columns': 'beer_name,style,brew_factory_name',
    'split_delimiter': '/'
}


In [67]:
df_c = dp.data_preprocessing([df1, df2], parameters)

print(df_c.shape)

df_c = df_c.drop('id', axis=1)

(7345, 5)


In [68]:
df_c.head()

,beer_name,brew_factory_name,style,abv
0,tr_egs_nugget_nectar,tr_egs_brewing_company,american_amber_red_ale,7.5
1,fat_tire_amber_ale,new_belgium_brewing,american_amber_red_ale,5.2
2,levitation_ale,stone_brewing_co,american_amber_red_ale,4.4
3,green_flash_hop_head_red_ale,green_flash_brewing_co,american_amber_red_ale,8.1
4,tr_egs_hopback_amber_ale,tr_egs_brewing_company,american_amber_red_ale,6


#### Generating the schema matching dataset in the heuristic case. 

In [6]:
parameters = {
    'output_file': 'beer',
    'concatenate': 'horizon',
    'missing_value': 'nan,ukn,none,unknown,',
    'missing_value_strategy': '',
    'round_number': 1,
    'round_columns': 'abv',
    'auto_merge': False,
    'tokenize_shared': True 
}

df_c = dp.data_preprocessing([df1.drop('id', axis=1), df2.drop('id', axis=1)], parameters)
# df_c = df_c.drop('id', axis=1)
df_c.to_csv('../pipeline/datasets/beer/beer-heuristic-sm.csv', index=False)

../EmbDI/data_preprocessing.py:120: UserWarning: Column abv is marked to be rounded, but it contains non-numeric characters.
  warnings.warn('Column {} is marked to be rounded, but it contains non-numeric characters.'.format(c))


#### Generating the schema matching dataset in the basic case. 

In [12]:
parameters = {
    'output_file': 'beer',
    'concatenate': 'horizon',
    'missing_value': 'nan,ukn,none,unknown,',
    'missing_value_strategy': '',
    'round_number': 1,
    'round_columns': 'abv',
    'auto_merge': False,
    'tokenize_shared': False 
}

df_c = dp.data_preprocessing([df1.drop('id', axis=1), df2.drop('id', axis=1)], parameters)
# df_c = df_c.drop('id', axis=1)
df_c.to_csv('../pipeline/datasets/beer/beer-master-sm.csv', index=False)

../EmbDI/data_preprocessing.py:117: UserWarning: Column abv is marked to be rounded, but it contains non-numeric characters.
  warnings.warn('Column {} is marked to be rounded, but it contains non-numeric characters.'.format(c))


In [11]:
lcol = len(df_c.columns)//2
for _ in range(lcol):
    print('{},{}'.format(df_c.columns[_],df_c.columns[_+lcol]))

0_abv,1_abv
0_beer_name,1_beer_name
0_brew_factory_name,1_brew_factory_name
0_style,1_style


#### Replace similar words, normalized edit distance used as metric

In [61]:
words = dp.get_unique_string_values(df_c, 'beer_name', 'word')
print(len(words))

6340


In [62]:
start_time = datetime.datetime.now()
lsh = dp.LSHMerge(list(words), 2, .5, 128)
replacement = lsh.get_replacement('normalized_edit_distance', 0.15)
end_time = datetime.datetime.now()
diff = end_time - start_time
print('Time taken: {}'.format(diff.total_seconds()))

Time taken: 10.298316


In [63]:
df_c = dp.merge(df_c, replacement, ['beer_name'], 'word')
df_c.head()

,beer_name,brew_factory_name,style,abv
0,tr_egs_nugget_nectar,tr_egs_brewing_company,american_amber_red_ale,7.5
1,NaN,new_belgium_brewing,american_amber_red_ale,5.2
2,NaN,stone_brewing_co,american_amber_red_ale,4.4
3,NaN,green_flash_brewing_co,american_amber_red_ale,8.1
4,NaN,tr_egs_brewing_company,american_amber_red_ale,6


#### Write the master dataset on file

In [69]:
df_c.to_csv('../pipeline/datasets/beer-master-updated-test.csv', index=False)

In [26]:
dp.write_info_file([df1, df2], 'info-beer.txt', [f1,f2])

#### Prepare the match file

In [28]:
import os
import os.path

tot_m = 0
dir_path = '../pipeline/experiments/beer/exp_data/'
with open('../pipeline/matches/matches-beer.txt', 'w') as fo:
    for file in os.listdir(dir_path):
        if file not in [os.path.basename(_) for _ in [f1, f2]]:        
            print(file)
            m = 0
            with open(dir_path + file, 'r') as fp:
                for idx, line in enumerate(fp):
                    m1, m2, flag = line.rstrip().rsplit(',')
                    if flag == '1':
                        s = 'idx_{0},idx_{1}\n'.format(m1, str(int(m2) + len(df1)))
                        fo.write(s)
                        m+=1
            print('File {}: {} matches.'.format(file, m))
            tot_m+=m
print('Total matches: {}'.format(tot_m))

test.csv
File test.csv: 14 matches.
train.csv
File train.csv: 40 matches.
valid.csv
File valid.csv: 14 matches.
Total matches: 68


In [34]:
df1_c = dp.data_preprocessing([df1], parameters)

df1_c = df1_c.drop('id', axis=1)

In [36]:
df2_c = dp.data_preprocessing([df2], parameters)

df2_c = df2_c.drop('id', axis=1)

In [42]:
l_df1 = set(df1_c.values.ravel())
l_df2 = set(df2_c.values.ravel())


In [45]:
intersect = l_df1.intersection(l_df2)